In [2]:
! pip install librosa
! apt-get install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.3.4-2).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [13]:
import os
import pandas as pd
import librosa
import glob 
import librosa.display
import numpy as np
from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical

In [0]:
! wget https://goo.gl/8hY5ER  
! tar -xzf 8hY5ER

In [12]:
def load_audio_files(base_dir,folders,files_per_folder=200):
  data = {}
  sampling_rates = []
  for folder in folders:
    dir_path = base_dir + folder
    files = os.listdir(dir_path)[0:files_per_folder]
    for file in files:
      file_path = dir_path + "/" + file
      try:
        audio_data, sampling_rate = librosa.load(file_path)
        data[file] = audio_data
        sampling_rates.append(sampling_rate)
      except Exception as e: 
        print("Error encountered while parsing file: ", file)
    print(folder, " completed")
  return data,sampling_rates

In [13]:
base_dir = "UrbanSound8K/audio/"
folders = os.listdir("UrbanSound8K/audio/")
folders.remove(".DS_Store")
data,s_rate = load_audio_files(base_dir,folders)
print(len(data))

fold3  completed
fold7  completed
fold1  completed
fold8  completed
fold10  completed
fold2  completed
fold4  completed
fold6  completed
fold9  completed
fold5  completed
2000


In [14]:
salience = {}
label = {}
df = pd.read_csv("UrbanSound8K/metadata/UrbanSound8K.csv")
for i in range(df.shape[0]):
  salience[df["slice_file_name"][i]] = df["salience"][i] - 1 # -1 to make it 0 or 1 labelled, currently it was 1 or 2
  label[df["slice_file_name"][i]] = df["classID"][i]
df = pd.DataFrame()
sal = []
lab = []
for key in data.keys():
  sal.append(salience[key])
  lab.append(label[key])

In [15]:
column_wise_values = []
values = list(data.values())
samples_len = len(values)
for x in range(88200):
  column = []
  for y in range(samples_len):
    if(x>values[y].shape[0]-1):
      column.append(0)
    else:
      column.append(values[y][x])
  column_wise_values.append(column)
len(column_wise_values)

88200

In [16]:
import numpy as np
np_arr = np.array(column_wise_values) # will make array of 88200 * 2000
np_arr = np.transpose(np_arr) # will make it in shape 2000 * 88200, each row represents one sample
df = pd.DataFrame(np_arr)
df["file_id"] = data.keys()
df["salience"] = sal
df["label"] = lab

In [18]:
file_to_store = "200each.pkl"
df.to_pickle(file_to_store)

In [22]:
! pip install boto3
import boto3
# Saving result with results of current model
session = boto3.Session(aws_access_key_id="45W63QC247Y23GVZDFEN",aws_secret_access_key="9rpr7dJCWRagc+gZyzKYmrXE2zuTzxzdCK8AxQQpJ7w")
s3 = session.resource('s3',endpoint_url="https://ams3.digitaloceanspaces.com",region_name="ams3")
bucket = s3.Bucket("swarnimtouch")
with open(file_to_store, 'rb') as data:
    bucket.upload_fileobj(data, file_to_store)
object_acl = s3.ObjectAcl('swarnimtouch',file_to_store)
response = object_acl.put(ACL='public-read')


    100% |████████████████████████████████| 133kB 3.9MB/s 
    100% |████████████████████████████████| 61kB 6.0MB/s 
    100% |████████████████████████████████| 4.4MB 5.8MB/s 
    100% |████████████████████████████████| 552kB 14.9MB/s 


In [20]:
df.info()

1.41124808

### Now we can simply download dataframe file without reading mp3/wav audio files

In [23]:
! wget https://swarnimtouch.ams3.digitaloceanspaces.com/200each.pkl

--2018-07-09 07:41:42--  https://swarnimtouch.ams3.digitaloceanspaces.com/200each.pkl
Resolving swarnimtouch.ams3.digitaloceanspaces.com (swarnimtouch.ams3.digitaloceanspaces.com)... 5.101.110.225
Connecting to swarnimtouch.ams3.digitaloceanspaces.com (swarnimtouch.ams3.digitaloceanspaces.com)|5.101.110.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1411890982 (1.3G) [binary/octet-stream]
Saving to: ‘200each.pkl.1’

200each.pkl.1        73%[=============>      ] 990.01M  26.2MB/s    eta 15s    200each.pkl.1       100%[===================>]   1.31G  26.6MB/s    in 54s     

2018-07-09 07:42:37 (25.1 MB/s) - ‘200each.pkl.1’ saved [1411890982/1411890982]



In [2]:
df = pd.read_pickle("200each.pkl")

In [3]:
cols = df.columns.tolist()
cols.remove("file_id")
cols.remove("label")

In [60]:
librosa_data_cols = list(cols)
librosa_data_cols.remove("salience")
new_features = []
for i in range(df.shape[0]):
  d = df.iloc[i][librosa_data_cols].values.astype("float")
  new_features.append(np.mean(librosa.feature.mfcc(y=d, sr=22050, n_mfcc=40).T,axis=0))

In [67]:
X = np.array(new_features)
Y = df["label"].iloc[:].values
Y = to_categorical(Y,num_classes=10)

In [68]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.25,random_state=0)

In [69]:
def create_model():
  model = Sequential()
  model.add(Dense(300,input_dim=40,activation="sigmoid"))
  model.add(Dropout(0.3))
  model.add(Dense(400,activation="sigmoid"))
  model.add(Dropout(0.3))
  model.add(Dense(300,activation="relu"))
  model.add(Dropout(0.3))
  model.add(Dense(200,activation="sigmoid"))
  model.add(Dropout(0.3))
  model.add(Dense(100,activation="sigmoid"))
  model.add(Dropout(0.3))
  model.add(Dense(10,activation="softmax"))
  
  model.compile(optimizer="adam",loss='categorical_crossentropy',metrics=['accuracy'])
  #model.summary()
  return model

In [72]:
model = create_model()
model.fit(X_train,Y_train,epochs=100,validation_data=(X_test,Y_test))

Train on 1500 samples, validate on 500 samples
Epoch 1/100
1500/1500 [==============================] - 3s 2ms/step - loss: 2.3814 - acc: 0.1053 - val_loss: 2.2813 - val_acc: 0.1060
Epoch 2/100
1500/1500 [==============================] - 1s 553us/step - loss: 2.2828 - acc: 0.1387 - val_loss: 2.2073 - val_acc: 0.1340
Epoch 3/100
1500/1500 [==============================] - 1s 454us/step - loss: 2.1312 - acc: 0.2013 - val_loss: 1.9389 - val_acc: 0.2560
Epoch 4/100
1500/1500 [==============================] - 1s 549us/step - loss: 1.9553 - acc: 0.2493 - val_loss: 1.7768 - val_acc: 0.3020
Epoch 5/100
1500/1500 [==============================] - 1s 534us/step - loss: 1.8199 - acc: 0.2887 - val_loss: 1.7222 - val_acc: 0.3220
Epoch 6/100
1500/1500 [==============================] - 1s 560us/step - loss: 1.7231 - acc: 0.3567 - val_loss: 1.7873 - val_acc: 0.3060
Epoch 7/100
1500/1500 [==============================] - 1s 536us/step - loss: 1.6982 - acc: 0.3373 - val_loss: 1.6214 - val_acc: 0.3

 768/1500 [==============>...............] - ETA: 0s - loss: 0.5770 - acc: 0.79951500/1500 [==============================] - 1s 578us/step - loss: 0.5832 - acc: 0.7987 - val_loss: 0.8816 - val_acc: 0.7340
Epoch 65/100
1500/1500 [==============================] - 1s 635us/step - loss: 0.5740 - acc: 0.8200 - val_loss: 0.9436 - val_acc: 0.7260
Epoch 66/100
1500/1500 [==============================] - 1s 576us/step - loss: 0.5644 - acc: 0.8133 - val_loss: 0.9532 - val_acc: 0.7260
Epoch 67/100
1500/1500 [==============================] - 1s 629us/step - loss: 0.5638 - acc: 0.8147 - val_loss: 0.9829 - val_acc: 0.7020
Epoch 68/100
1500/1500 [==============================] - 1s 544us/step - loss: 0.5366 - acc: 0.8333 - val_loss: 0.9437 - val_acc: 0.7180
Epoch 69/100
1500/1500 [==============================] - 1s 555us/step - loss: 0.5735 - acc: 0.8053 - val_loss: 0.8920 - val_acc: 0.7260
Epoch 70/100
1500/1500 [==============================] - 1s 665us/step - loss: 0.4911 - acc: 0.8407 - v

In [78]:
! cd /etc ; ls

adduser.conf		host.conf      mime.types     rc6.d
alternatives		hostname       mke2fs.conf    rcS.d
apt			hosts	       mtab	      resolv.conf
bash.bashrc		hosts.allow    networks       rmt
bash_completion.d	hosts.deny     nsswitch.conf  securetty
bindresvport.blacklist	init.d	       openal	      security
ca-certificates		inputrc        opt	      selinux
ca-certificates.conf	ipython        os-release     sensors3.conf
cron.daily		issue	       pam.conf       sensors.d
cron.weekly		issue.net      pam.d	      shadow
dbus-1			jupyter        passwd	      shadow-
debconf.conf		kernel	       passwd-	      shells
debian_version		ldap	       perl	      skel
default			ld.so.cache    pip.conf       ssh
deluser.conf		ld.so.conf     profile	      ssl
dpkg			ld.so.conf.d   profile.d      subgid
drirc			legal	       pulse	      subuid
environment		libaudit.conf  python	      sysctl.conf
ffserver.conf		locale.alias   python2.7      sysctl.d
fonts			locale.gen     python3	      systemd
fstab			localtime